In [19]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../results/result.csv')

repo_num = 3042
# merge_tool = "gitmerge-recursive-patience"
# merge_tool = "gitmerge-recursive-minimal"
merge_tool = "spork"
# merge_tool = "gitmerge-resolve"
# merge_tool = "intellimerge"
# merge_tool = "gitmerge-ort-ignorespace"
# merge_tool = "gitmerge-ort"

script = "../scripts/merge_tools/" + merge_tool + ".sh"

In [20]:
from merge_tester import MERGE_STATE
df[merge_tool].apply(lambda x: MERGE_STATE[x])[0]

<MERGE_STATE.Tests_passed: 5>

In [21]:
from validate_repos import clone_repo
repo = clone_repo(df.iloc[repo_num]["repo_name"]) # Return a Git-Python repo object

jamesdbloom/mockserver  : Cloning repo
Repository 'jamesdbloom/mockserver' cloned to path: repos/jamesdbloom/mockserver
jamesdbloom/mockserver  : Finished cloning


In [22]:
# For a git to work you need to pass a name for each branch
repo.remote().fetch()
repo.git.checkout(df.iloc[repo_num]["left"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "TEMP_LEFT_BRANCH", force=True)
repo.git.checkout(df.iloc[repo_num]["right"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "TEMP_RIGHT_BRANCH", force=True)

''

In [23]:
repo_path = repo.git.rev_parse("--show-toplevel")
repo_path

'/home/benrr/miniconda3/envs/merge_1/AST-Merging-Evaluation/src/python/repos/jamesdbloom/mockserver'

In [24]:
import subprocess
# Returns = 1 process fails
subprocess.call([script,repo_path,"TEMP_LEFT_BRANCH","TEMP_RIGHT_BRANCH"])

Switched to branch 'TEMP_LEFT_BRANCH'


Auto-merging mockserver-core/src/main/java/org/mockserver/socket/tls/bouncycastle/BCKeyAndCertificateFactory.java
Merge made by the 'ort' strategy.
 .../org/mockserver/client/MockServerClient.java    |   4 +-
 .../java/org/mockserver/socket/tls/PEMToFile.java  |  15 ++-
 .../bouncycastle/BCKeyAndCertificateFactory.java   | 112 +++++++--------------
 .../mtls/generate_custom_certifcates.md            |   1 -
 .../mtls/generate_custom_certificates.md           |   1 +
 ...gMTLSClientNotAuthenticatedIntegrationTest.java |   4 +-
 ...ertificateWithECKeysMockingIntegrationTest.java |  59 +++++++++++
 .../netty/integration/tls/ec/ca-csr.json           |  14 +++
 .../netty/integration/tls/ec/ca-key-pkcs8.pem      |   5 +
 .../mockserver/netty/integration/tls/ec/ca-key.pem |   5 +
 .../org/mockserver/netty/integration/tls/ec/ca.pem |  11 ++
 .../mockserver/netty/integration/tls/ec/csr.json   |  21 ++++
 .../tls/ec/generate_custom_certificates.md         |  11 ++
 .../netty/integration/tls/ec/l

0

In [25]:
subprocess.call(["cd",repo_path,"&&","git","merge","--abort"])

FileNotFoundError: [Errno 2] No such file or directory: 'cd'

In [ ]:
# Merge Sucess (Return 0)
subprocess.call(["../scripts/merge_tools/gitmerge-ort-ignorespace.sh",repo_path,"NAME_OF_LEFT_BRANCH","NAME_OF_RIGHT_BRANCH"])
# Be careful the merge has been completed the history of the left branch has now changed and you can't 
# merge again unless you delete the repo and clone it again